In [84]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = np.array(iris['data'])
y = np.array(iris['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape)

(120, 4) (120,)


In [85]:
num_features = x.shape[1]

def gini_impurity(y):
    unique_counts = np.unique_counts(y).counts
    return 1 - np.sum(np.square(unique_counts / np.sum(unique_counts)))

def weighted_gini_impurity(y_left, y_right):
    left_impurity = gini_impurity(y_left)
    right_impurity = gini_impurity(y_right)
    size = y_left.shape[0] + y_right.shape[0]

    return y_left.shape[0] / size * left_impurity + y_right.shape[0] / size * right_impurity


In [87]:
class Node:
    def __init__(self):
        self.x = None
        self.y = None
        self.left = None
        self.right = None
        self.decision_feature_index = None
        self.decision_threshold = None
        self.impurity = None

    def fit(self, x, y):
        self.x = x
        self.y = y
        self.impurity = weighted_gini_impurity(self.y, self.x)
        feature_index, threshold, impurity = self._find_best_feature_to_split()

        if impurity >= self.impurity:
            return

        self.impurity = impurity
        self.decision_feature_index = feature_index
        self.decision_threshold = threshold
        # print(f"Decision Tree Node Split using feature {feature_index} with threshold {threshold} and impurity {impurity}")

        x_left = self.x[self.x[:, self.decision_feature_index] <= threshold]
        x_right = self.x[self.x[:, self.decision_feature_index] > threshold]
        y_left = self.y[self.x[:, self.decision_feature_index] <= threshold]
        y_right = self.y[self.x[:, self.decision_feature_index] > threshold]

        self.left = Node()
        self.right = Node()

        self.left.fit(x_left, y_left)
        self.right.fit(x_right, y_right)

    def predict(self, x):
        preds = []
        for example in x:
            preds.append(self._predict_example(example))
        return preds

    def _predict_example(self, x):
        if self.left is None or self.right is None:
            return np.argmax(np.bincount(self.y))

        if x[self.decision_feature_index] <= self.decision_threshold:
            return self.left._predict_example(x)
        else:
            return self.right._predict_example(x)


    def _find_best_feature_to_split(self):
        best_impurity = 1
        best_threshold = 0
        best_feature_index = 0

        for feature_index in range(self.x.shape[1]):
            feature_values = np.sort(self.x[:, feature_index])
            thresholds = [(feature_values[i] + feature_values[i + 1]) / 2 for i in range(feature_values.size - 1) if feature_values[i] != feature_values[i + 1]]

            for threshold in thresholds:
                y_left = self.y[self.x[:, feature_index] <= threshold]
                y_right = self.y[self.x[:, feature_index] > threshold]

                impurity = weighted_gini_impurity(y_left, y_right)
                if best_impurity > impurity:
                    best_impurity = impurity
                    best_threshold = threshold
                    best_feature_index = feature_index

        return best_feature_index, best_threshold, best_impurity

root = Node()
root.fit(x_train, y_train)
preds = root.predict(x_test)
print(preds, y_test)


[np.int64(1), np.int64(0), np.int64(2), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(2), np.int64(1), np.int64(1), np.int64(2), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(2), np.int64(1), np.int64(1), np.int64(2), np.int64(0), np.int64(2), np.int64(0), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(0), np.int64(0)] [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
